In [ ]:
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score

In [ ]:
stock = yf.Ticker("AAPL")
stock = stock.history(period='2y')


In [ ]:
stock

In [ ]:
del stock['Dividends']
del stock['Stock Splits']

In [ ]:
stock

In [ ]:
stock.plot.line(y="Close")

In [ ]:
stock["Next Days Close"] = stock['Close'].shift(-1)

In [ ]:
stock

In [ ]:
stock['change'] = (stock['Next Days Close'] > stock['Close']).astype(int)

In [ ]:
stock

In [ ]:
# Drop rows with NaN target values
stock = stock.dropna(subset=['Next Days Close', 'change'])

# Split our preprocessed data into our features and target arrays
y = stock['change']
X = stock.drop(columns=['change'])

# Preserve the index of the original DataFrame
original_index = X.index

In [ ]:
#Normalize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
#split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)
# Rebuild X_test DataFrame to restore index information
X_test_df = pd.DataFrame(X_test, index=original_index[-len(X_test):], columns=X.columns)

In [ ]:
# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_dim=X_train.shape[1]),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # For binary classification
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_df, y_test)
print(f'Test Loss: {model_loss}, Test Accuracy: {model_accuracy}')

In [ ]:
# Make predictions
predictions = model.predict(X_test_df).round().flatten()

In [ ]:
# Create a DataFrame for predictions with the index of X_test_df
predictions_df = pd.DataFrame(predictions, index=X_test_df.index, columns=['Predictions'])

In [ ]:
precision_score(y_test,predictions)

In [ ]:
# Combine actual values and predictions
combined = pd.concat([y_test, predictions_df], axis=1)

# Plot the combined DataFrame
combined.plot(title='Actual vs. Predicted')

In [ ]:
predictions_df.value_counts()